In [204]:
import findspark
findspark.init()

In [205]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Dfapp").getOrCreate()

In [145]:
df=spark.read.option("header","true").csv("F:\\datasets\\pitem.csv",header=True)
df.show()

+------+---------+--------+------+
|itemid| itemname|itemcost|sup_id|
+------+---------+--------+------+
|    11|    chalk|    1000|     1|
|    12|   Pencil|     900|     1|
|    13|      Pen|    1500|     2|
|    14|   Duster|    1600|     3|
|    15|    Scale|     100|     2|
|    16|    chalk|     600|     5|
|  null|     null|    1200|     6|
|    17|penpencil|      10|  null|
|    18|    Brush|     250|     2|
+------+---------+--------+------+



In [147]:
dfs=spark.read.option("header","true").csv("F:\\datasets\\supplier.csv",header=True)
dfs.show()

+------+--------+
|sup_id|Sup_name|
+------+--------+
|     1|       X|
|     2|       Y|
|     3|       X|
|     4|       A|
|     5|    null|
|     6|       B|
|  null|    null|
|     7|       Q|
+------+--------+



In [148]:

df.createOrReplaceTempView("tempview")
dfs.createOrReplaceTempView("tempviewsup")

In [149]:
res=spark.sql("select * from tempview")
r=spark.sql("select * from tempviewsup")

In [150]:
res.show()

+------+---------+--------+------+
|itemid| itemname|itemcost|sup_id|
+------+---------+--------+------+
|    11|    chalk|    1000|     1|
|    12|   Pencil|     900|     1|
|    13|      Pen|    1500|     2|
|    14|   Duster|    1600|     3|
|    15|    Scale|     100|     2|
|    16|    chalk|     600|     5|
|  null|     null|    1200|     6|
|    17|penpencil|      10|  null|
|    18|    Brush|     250|     2|
+------+---------+--------+------+



In [151]:
r.show()

+------+--------+
|sup_id|Sup_name|
+------+--------+
|     1|       X|
|     2|       Y|
|     3|       X|
|     4|       A|
|     5|    null|
|     6|       B|
|  null|    null|
|     7|       Q|
+------+--------+



In [152]:
#1.show item details supplied by MR.X
r1=spark.sql("select * from tempview as p join tempviewsup as p1 on p.sup_id=p1.sup_id where p1.Sup_name='X'")
r1.show()

+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|Sup_name|
+------+--------+--------+------+------+--------+
|    11|   chalk|    1000|     1|     1|       X|
|    12|  Pencil|     900|     1|     1|       X|
|    14|  Duster|    1600|     3|     3|       X|
+------+--------+--------+------+------+--------+



In [203]:
#2.show supplier details who has supplied items cost>1000
r2=spark.sql("select * from tempview where itemcost>1000")
r2.show()

+------+--------+--------+------+
|itemid|itemname|itemcost|sup_id|
+------+--------+--------+------+
|    13|     Pen|    1500|     2|
|    14|  Duster|    1600|     3|
|  null|    null|    1200|     6|
+------+--------+--------+------+



In [198]:
#3.show all item details whos supplier details are not available
r4=spark.sql("select * from tempview as p join tempviewsup as p1 on p.sup_id==p1.sup_id where p1.Sup_name is null ")
r4.show()


+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|Sup_name|
+------+--------+--------+------+------+--------+
|    16|   chalk|     600|     5|     5|    null|
+------+--------+--------+------+------+--------+



In [155]:
#4.show item details whos supplier details are available
r4=spark.sql("select * from tempview as p join tempviewsup as p1 on p.sup_id=p1.sup_id where p1.sup_id==p.sup_id and p1.Sup_name!='null'")
r4.show()

+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|Sup_name|
+------+--------+--------+------+------+--------+
|    11|   chalk|    1000|     1|     1|       X|
|    12|  Pencil|     900|     1|     1|       X|
|    13|     Pen|    1500|     2|     2|       Y|
|    14|  Duster|    1600|     3|     3|       X|
|    15|   Scale|     100|     2|     2|       Y|
|  null|    null|    1200|     6|     6|       B|
|    18|   Brush|     250|     2|     2|       Y|
+------+--------+--------+------+------+--------+



In [199]:
#5show item details along with supplier details for such items,for which supplier details 
#are available and item name starts with 'b'

r5=spark.sql("select * from tempview as p join tempviewsup as p1 on p.sup_id=p1.sup_id where   p.itemname like 'B%'")
r5.show()

+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|Sup_name|
+------+--------+--------+------+------+--------+
|    18|   Brush|     250|     2|     2|       Y|
+------+--------+--------+------+------+--------+



In [201]:
#6.supplier wise nor of items supplied,sum of all items supplied by particular supplier,min ,max total of 
#all item cost supplied
r6=spark.sql("select count(itemid),min(itemcost) ,max(itemcost),sum(itemcost)from tempview" )
r6.show()


+-------------+-------------+-------------+-------------+
|count(itemid)|min(itemcost)|max(itemcost)|sum(itemcost)|
+-------------+-------------+-------------+-------------+
|            8|           10|          900|       7160.0|
+-------------+-------------+-------------+-------------+



In [192]:
#join overall items available in 2 stores
i=df.join(dfs,df.sup_id==dfs.sup_id,"inner")
i.show()


+------+--------+--------+------+------+--------+
|itemid|itemname|itemcost|sup_id|sup_id|Sup_name|
+------+--------+--------+------+------+--------+
|    11|   chalk|    1000|     1|     1|       X|
|    12|  Pencil|     900|     1|     1|       X|
|    13|     Pen|    1500|     2|     2|       Y|
|    14|  Duster|    1600|     3|     3|       X|
|    15|   Scale|     100|     2|     2|       Y|
|    16|   chalk|     600|     5|     5|    null|
|  null|    null|    1200|     6|     6|       B|
|    18|   Brush|     250|     2|     2|       Y|
+------+--------+--------+------+------+--------+



In [208]:
pdf=spark.read.csv("F:\\employees.csv",header=True,inferSchema=True)
pdf.show()

+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|                Team|
+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|   Douglas|  Male|      null|           null|  null|   null|             null|           Marketing|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|                null|
|     Maria|  null| 4/23/1993|       11:17 AM|  null| 11.858|            false|             Finance|
|     Jerry|  Male|03-04-2005|           null|  null|   null|             true|             Finance|
|     Larry|  Male| 1/24/1998|        4:47 PM|  null|  1.389|             true|     Client Services|
|    Dennis|  null| 4/18/1987|        1:35 AM|115163| 10.125|            false|               Legal|
|      Ruby|Female|      null|           null| 65476| 10.012|             true|            

In [209]:
pdf.printSchema()

root
 |-- First Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Last Login Time: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Bonus %: double (nullable = true)
 |-- Senior Management: boolean (nullable = true)
 |-- Team: string (nullable = true)



In [348]:
from pyspark.sql.functions import col
pdf1=pdf.select(col('First Name').alias("firstname"),col('Gender').alias("Gen"),col('Start Date').alias('StartDate'),
               col('Last Login Time').alias('LastLoginTime'),col('Salary').alias('Sal'),col('Bonus %').alias('Bonus'),
                col('Senior Management').alias('SM'),col('Team').alias('Team'))
pdf1.show()

+---------+------+----------+-------------+------+------+-----+--------------------+
|firstname|   Gen| StartDate|LastLoginTime|   Sal| Bonus|   SM|                Team|
+---------+------+----------+-------------+------+------+-----+--------------------+
|  Douglas|  Male|      null|         null|  null|  null| null|           Marketing|
|   Thomas|  Male| 3/31/1996|      6:53 AM| 61933|  4.17| true|                null|
|    Maria|  null| 4/23/1993|     11:17 AM|  null|11.858|false|             Finance|
|    Jerry|  Male|03-04-2005|         null|  null|  null| true|             Finance|
|    Larry|  Male| 1/24/1998|      4:47 PM|  null| 1.389| true|     Client Services|
|   Dennis|  null| 4/18/1987|      1:35 AM|115163|10.125|false|               Legal|
|     Ruby|Female|      null|         null| 65476|10.012| true|             Product|
|     null|Female| 7/20/2015|     10:43 AM|  null|  null| null|             Finance|
|   Angela|Female|      null|      6:29 AM| 95570|  null| true|  

In [349]:
pdf1.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- Gen: string (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- LastLoginTime: string (nullable = true)
 |-- Sal: integer (nullable = true)
 |-- Bonus: double (nullable = true)
 |-- SM: boolean (nullable = true)
 |-- Team: string (nullable = true)



In [272]:
pdf1.createOrReplaceTempView("tempview")


In [273]:
res=spark.sql("select * from tempview")
res.show()

+------+--------+--------+---------+-----+----+
|itemid|itemname|itemcost|supplierd|Grade| Qyt|
+------+--------+--------+---------+-----+----+
|     8|   chalk|   100.0|        X|    A|  10|
|    12|  Pencil|    50.0|        Y|    B|   5|
|    13|    null|    null|     null| null|null|
|    15|  Duster|    20.0|        Y|    C|   4|
|    23|   Scale|    null|     null|    A|   3|
|    34|  Colors|    50.0|     null|    B|   1|
+------+--------+--------+---------+-----+----+



In [346]:
#r1=spark.sql("select Team,sum(Sal) as total_Sal from tempview GROUP BY Team").show()


In [345]:
from pyspark.ml.feature import SQLTransformer

In [236]:
res=SQLTransformer(statement="select firstname,Team,Sal,Bonus from __THIS__")
res.transform(pdf1).show()

+---------+--------------------+------+------+
|firstname|                Team|   Sal| Bonus|
+---------+--------------------+------+------+
|  Douglas|           Marketing|  null|  null|
|   Thomas|                null| 61933|  4.17|
|    Maria|             Finance|  null|11.858|
|    Jerry|             Finance|  null|  null|
|    Larry|     Client Services|  null| 1.389|
|   Dennis|               Legal|115163|10.125|
|     Ruby|             Product| 65476|10.012|
|     null|             Finance|  null|  null|
|   Angela|         Engineering| 95570|  null|
|  Frances|Business Development|139852| 7.524|
|   Louise|                null| 63241|15.132|
|    Julie|               Legal|102508|12.637|
|  Brandon|     Human Resources|112807|17.492|
|     Gary|               Sales|109831| 5.831|
| Kimberly|             Finance| 41426|14.543|
|  Lillian|             Product| 59414| 1.256|
|   Jeremy|     Human Resources| 90370| 7.369|
|    Shawn|             Product|111737| 6.414|
|    Diana|  

In [265]:
item=spark.read.csv("F:\\item_details.csv",header=True,inferSchema=True)
item.show()
item.printSchema()

+-------+---------+---------+--------+-----+----+
|item id|item name|item cost|supplier|grade| Qyt|
+-------+---------+---------+--------+-----+----+
|      8|    chalk|      100|       X|    A|  10|
|     12|   Pencil|       50|       Y|    B|   5|
|     13|     null|     null|    null| null|null|
|     15|   Duster|       20|       Y|    C|   4|
|     23|    Scale|     null|    null|    A|   3|
|     34|   Colors|       50|    null|    B|   1|
+-------+---------+---------+--------+-----+----+

root
 |-- item id: integer (nullable = true)
 |-- item name: string (nullable = true)
 |-- item cost: integer (nullable = true)
 |-- supplier: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- Qyt: integer (nullable = true)



In [266]:
from pyspark.sql.functions import col
pdfi=item.select(col('item id').alias("itemid"),col('item name').alias("itemname"),col('item cost').alias('itemcost'),
               col('supplier').alias('supplierd'),col('grade').alias('Grade'),col('Qyt').alias('Qyt'),
        )
pdfi.show()

+------+--------+--------+---------+-----+----+
|itemid|itemname|itemcost|supplierd|Grade| Qyt|
+------+--------+--------+---------+-----+----+
|     8|   chalk|     100|        X|    A|  10|
|    12|  Pencil|      50|        Y|    B|   5|
|    13|    null|    null|     null| null|null|
|    15|  Duster|      20|        Y|    C|   4|
|    23|   Scale|    null|     null|    A|   3|
|    34|  Colors|      50|     null|    B|   1|
+------+--------+--------+---------+-----+----+



In [245]:
res=SQLTransformer(statement="select min(itemcost),max(itemcost) from __THIS__")
res.transform(pdf1).show()

+-------------+-------------+
|min(itemcost)|max(itemcost)|
+-------------+-------------+
|          4.0|        65.76|
+-------------+-------------+



In [248]:
from pyspark.sql.functions  import expr
pdf1.withColumn("incre",expr("itemcost+20")).show()

+------+--------+--------+---------+-----+----+-----+
|itemid|itemname|itemcost|supplierd|Grade| Qyt|incre|
+------+--------+--------+---------+-----+----+-----+
|     8|   chalk|   65.76|        X|    A|  10|85.76|
|    12|  Pencil|     4.0|        Y|    B|   5| 24.0|
|    13|    null|    null|     null| null|null| null|
|    15|  Duster|    54.0|        Y|    C|   4| 74.0|
|    23|   Scale|    null|     null|    A|   3| null|
|    34|  Colors|    50.0|     null|    B|   1| 70.0|
+------+--------+--------+---------+-----+----+-----+



In [249]:
pdf1.selectExpr('itemid','itemcost','itemcost+20').show()

+------+--------+---------------+
|itemid|itemcost|(itemcost + 20)|
+------+--------+---------------+
|     8|   65.76|          85.76|
|    12|     4.0|           24.0|
|    13|    null|           null|
|    15|    54.0|           74.0|
|    23|    null|           null|
|    34|    50.0|           70.0|
+------+--------+---------------+



In [255]:
pdf1.withColumn("Gen",expr("CASE WHEN Gen='Male' THEN 'M'" + "WHEN Gen='Female' THEN 'F' END ")).show()

+---------+----+----------+-------------+------+------+-----+--------------------+
|firstname| Gen| StartDate|LastLoginTime|   Sal| Bonus|   SM|                Team|
+---------+----+----------+-------------+------+------+-----+--------------------+
|  Douglas|   M|      null|         null|  null|  null| null|           Marketing|
|   Thomas|   M| 3/31/1996|      6:53 AM| 61933|  4.17| true|                null|
|    Maria|null| 4/23/1993|     11:17 AM|  null|11.858|false|             Finance|
|    Jerry|   M|03-04-2005|         null|  null|  null| true|             Finance|
|    Larry|   M| 1/24/1998|      4:47 PM|  null| 1.389| true|     Client Services|
|   Dennis|null| 4/18/1987|      1:35 AM|115163|10.125|false|               Legal|
|     Ruby|   F|      null|         null| 65476|10.012| true|             Product|
|     null|   F| 7/20/2015|     10:43 AM|  null|  null| null|             Finance|
|   Angela|   F|      null|      6:29 AM| 95570|  null| true|         Engineering|
|  F

In [271]:
pdfi.withColumn("Grade",expr("CASE WHEN itemcost>=100 THEN 'A'" + 'WHEN itemcost<=50  THEN "B" else "C" END')).show()

+------+--------+--------+---------+-----+----+
|itemid|itemname|itemcost|supplierd|Grade| Qyt|
+------+--------+--------+---------+-----+----+
|     8|   chalk|     100|        X|    A|  10|
|    12|  Pencil|      50|        Y|    B|   5|
|    13|    null|    null|     null|    C|null|
|    15|  Duster|      20|        Y|    B|   4|
|    23|   Scale|    null|     null|    C|   3|
|    34|  Colors|      50|     null|    B|   1|
+------+--------+--------+---------+-----+----+



In [287]:
pdf1.select('firstname','Sal','Bonus',expr('round(Bonus,2) as roundup')).show(3)

+---------+-----+------+-------+
|firstname|  Sal| Bonus|roundup|
+---------+-----+------+-------+
|  Douglas| null|  null|   null|
|   Thomas|61933|  4.17|   4.17|
|    Maria| null|11.858|  11.86|
+---------+-----+------+-------+
only showing top 3 rows



In [289]:
pdf1.selectExpr('firstname','Sal','Bonus',('round(Bonus,1) as roundup')).show(2)

+---------+-----+-----+-------+
|firstname|  Sal|Bonus|roundup|
+---------+-----+-----+-------+
|  Douglas| null| null|   null|
|   Thomas|61933| 4.17|    4.2|
+---------+-----+-----+-------+
only showing top 2 rows



In [291]:
pdf1.selectExpr("firstname","round(Bonus,1)").filter('firstname=="Jerry"').toPandas

,firstname,"round(Bonus, 1)"
0,Jerry,NaN
1,Jerry,19.1
2,Jerry,9.2
3,Jerry,18.8
4,Jerry,11.4
5,Jerry,18.9



pdf1.selectExpr("firstname","Team","round(Bonus,1)").filter('length(Team)=="7"').show()

In [316]:

r1=pdf1.groupBy("Team").sum("Sal").show()

+--------------------+--------+
|                Team|sum(Sal)|
+--------------------+--------+
|               Sales| 8664303|
|         Engineering| 8532753|
|                null| 4042828|
|Business Development| 9278498|
|             Finance| 9091186|
|     Client Services| 9250785|
|        Distribution| 7965042|
|               Legal| 7858718|
|           Marketing| 8765380|
|             Product| 8423223|
|     Human Resources| 8275952|
+--------------------+--------+



In [331]:
pdf1.selectExpr("Team","Sal").filter('length(Team)=="7"').groupBy("Team").sum('Sal').show()

+-------+--------+
|   Team|sum(Sal)|
+-------+--------+
|Finance| 9091186|
|Product| 8423223|
+-------+--------+



In [325]:
r1=pdf1.groupBy("Team").sum("Sal")
r1.show()

+--------------------+--------+
|                Team|sum(Sal)|
+--------------------+--------+
|               Sales| 8664303|
|         Engineering| 8532753|
|                null| 4042828|
|Business Development| 9278498|
|             Finance| 9091186|
|     Client Services| 9250785|
|        Distribution| 7965042|
|               Legal| 7858718|
|           Marketing| 8765380|
|             Product| 8423223|
|     Human Resources| 8275952|
+--------------------+--------+



In [354]:

#r=spark.sql('select Team,sum(Sal) from tempview where length(Team)==7 GROUP BY Team').show()